In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv',encoding= 'unicode_escape')

In [4]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [11]:
dt = data.drop(['emotion_in_tweet_is_directed_at'],axis=1)

In [12]:
dt.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [13]:
dt.columns = ["Text", "Label"]
dt.head()

,Text,Label
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [14]:
dt['Label'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: Label, dtype: int64

In [15]:
dt['Label']= dt['Label'].map({'No emotion toward brand or product':'neutral','Positive emotion':'positive','Negative emotion':'negative',"I can't tell":'no idea'})
dt['Label'] = dt['Label'].map({'neutral':2,'positive':3,'negative':1,'no idea':0})

In [16]:
dt.head()

,Text,Label
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,1
1,@jessedee Know about @fludapp ? Awesome iPad/i...,3
2,@swonderlin Can not wait for #iPad 2 also. The...,3
3,@sxsw I hope this year's festival isn't as cra...,1
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,3


In [17]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9092 non-null   object
 1   Label   9093 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 142.2+ KB


In [18]:
dt = dt.dropna()

In [19]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9092 entries, 0 to 9092
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9092 non-null   object
 1   Label   9092 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 213.1+ KB


In [20]:
import gensim
dt['text_clean_gensim'] = dt['Text'].apply(lambda x: gensim.utils.simple_preprocess(x))
dt.head()

,Text,Label,text_clean_gensim
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,1,"[wesley, have, iphone, after, hrs, tweeting, a..."
1,@jessedee Know about @fludapp ? Awesome iPad/i...,3,"[jessedee, know, about, fludapp, awesome, ipad..."
2,@swonderlin Can not wait for #iPad 2 also. The...,3,"[swonderlin, can, not, wait, for, ipad, also, ..."
3,@sxsw I hope this year's festival isn't as cra...,1,"[sxsw, hope, this, year, festival, isn, as, cr..."
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,3,"[sxtxstate, great, stuff, on, fri, sxsw, maris..."


In [21]:
from keras.preprocessing import text
tokenizer = text.Tokenizer() 
tokenizer.fit_on_texts(list(dt['text_clean_gensim']))
tokenized_texts = tokenizer.texts_to_sequences(dt['text_clean_gensim'])

In [23]:
from keras.utils import pad_sequences
X = pad_sequences(tokenized_texts, maxlen=100)
len(tokenizer.word_index)

9283

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dt['Label'].values, test_size=0.2)

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, SimpleRNN, Dropout

In [26]:
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index)+1, output_dim = 128,input_length=100))
model.add(SimpleRNN(10))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

In [27]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1188352   
                                                                 
 simple_rnn (SimpleRNN)      (None, 10)                1390      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 50)                550       
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 204       
                                                                 
Total params: 1,190,496
Trainable params: 1,190,496
Non-

In [29]:
model.fit(X_train, y_train, epochs=15, validation_split=0.1)

Epoch 1/15
205/205 [==============================] - 10s 36ms/step - loss: 1.0763 - accuracy: 0.5170 - val_loss: 0.9214 - val_accuracy: 0.5920
Epoch 2/15
205/205 [==============================] - 7s 36ms/step - loss: 0.9008 - accuracy: 0.5922 - val_loss: 0.8792 - val_accuracy: 0.5934
Epoch 3/15
205/205 [==============================] - 10s 48ms/step - loss: 0.7901 - accuracy: 0.6576 - val_loss: 0.8421 - val_accuracy: 0.6236
Epoch 4/15
205/205 [==============================] - 7s 35ms/step - loss: 0.6527 - accuracy: 0.7384 - val_loss: 0.8808 - val_accuracy: 0.6429
Epoch 5/15
205/205 [==============================] - 7s 35ms/step - loss: 0.5700 - accuracy: 0.7895 - val_loss: 0.9483 - val_accuracy: 0.6195
Epoch 6/15
205/205 [==============================] - 7s 35ms/step - loss: 0.4932 - accuracy: 0.8200 - val_loss: 1.0182 - val_accuracy: 0.6003
Epoch 7/15
205/205 [==============================] - 7s 34ms/step - loss: 0.4481 - accuracy: 0.8443 - val_loss: 1.0411 - val_accuracy: 0.62

In [31]:
y_pred = model.predict(X_test)

57/57 [==============================] - 1s 10ms/step


In [32]:
test1 = "its very good"
test1 = tokenizer.texts_to_sequences([test1])
test2 = pad_sequences(test1, maxlen=100)
out = model.predict(test2)
out

1/1 [==============================] - 0s 27ms/step


array([[0.02204014, 0.05280701, 0.84632695, 0.07882592]], dtype=float32)

In [33]:
test1 = "its very bad"
test1 = tokenizer.texts_to_sequences([test1])
test2 = pad_sequences(test1, maxlen=100)
out = model.predict(test2)
out

1/1 [==============================] - 0s 23ms/step


array([[7.9250295e-04, 4.8620455e-04, 9.9405849e-01, 4.6627834e-03]],
      dtype=float32)

In [34]:
test1 = "Sharing the link"
test1 = tokenizer.texts_to_sequences([test1])
test2 = pad_sequences(test1, maxlen=100)
out = model.predict(test2)
out

1/1 [==============================] - 0s 20ms/step


array([[1.4980029e-03, 5.4669630e-04, 9.9233305e-01, 5.6221825e-03]],
      dtype=float32)